In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def read_goog_sp500_dataframe():
    googFile = 'data/GOOG.csv'
    spFile = 'data/SP_500.csv'
    
    goog = pd.read_csv(googFile,sep=',',usecols=[0,5],names=['Date','Goog'],header=0)
    sp = pd.read_csv(spFile,sep=',',usecols=[0,5],names=['Date','SP500'],header=0)
    
    goog['SP500'] = sp['SP500']
    
    goog['Date'] = pd.to_datetime(goog['Date'],format='%Y-%m-%d')
    
    goog = goog.sort_values(['Date'],ascending=True)
    
    returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64','int64']]].pct_change()
    
    return returns

In [3]:
def read_goog_sp500_logistic_data():
    returns = read_goog_sp500_dataframe()
    returns['Intercept'] =1
    xData = np.array(returns[["SP500",  "Intercept"]][1:-1])
    yData = (returns["Goog"]>0)[1:-1]
    return (xData, yData)

In [4]:
xData, yData = read_goog_sp500_logistic_data()

In [5]:
features = [tf.contrib.layers.real_valued_column("x",dimension=1)]

In [16]:
estimator = tf.contrib.learn.LinearClassifier(feature_columns=features)

Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C124BFAF28>, '_device_fn': None, '_save_checkpoints_secs': 600, '_environment': 'local', '_model_dir': 'C:\\Users\\Lolly\\AppData\\Local\\Temp\\tmpgm3m85i5', '_task_id': 0, '_session_config': None, '_num_worker_replicas': 0, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_is_chief': True, '_log_step_count_steps': 100, '

In [17]:
x = np.expand_dims(xData[:,0],axis=1)
y = np.expand_dims(np.array(yData),axis=1)

In [18]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x},y,batch_size=100,num_epochs=10000)

In [20]:
fit = estimator.fit(input_fn=input_fn,steps=10000)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Lolly\AppData\Local\Temp\tmpgm3m85i5\model.ckpt.
INFO:tensorflow:loss = 0.6931472, step = 1
INFO:tensorflow:global_step/sec: 377.57
INFO:tensorflow:loss = 0.6890389, step = 101 (0.270 sec)
INFO:tensorflow:global_step/sec: 238.212
INFO:tensorflow:loss = 0.67718726, step = 201 (0.415 sec)
INFO:tensorflow:global_step/sec: 513.14
INFO:tensorflow:loss = 0.66086066, step = 301 (0.195 sec)
INFO:tensorflow:global_step/sec: 645.507
INFO:tensorflow:loss = 0.67423517, step = 401 (0.160 sec)
INFO:tensorflow:global_step/sec: 513.097
INFO:tensorflow:loss = 0.6753382, step = 501 (0.190 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 435.012
INFO:tensorflow:loss = 0.58565444, step = 7401 (0.210 sec)
INFO:tensorflow:global_step/sec: 402.9
INFO:tensorflow:loss = 0.54509556, step = 7501 (0.248 sec)
INFO:tensorflow:global_step/sec: 571.765
INFO:tensorflow:loss = 0.51124877, step = 7601 (0.175 sec)
INFO:tensorflow:global_step/sec: 513.097
INFO:tensorflow:loss = 0.5732865, step = 7701 (0.190 sec)
INFO:tensorflow:global_step/sec: 526.597
INFO:tensorflow:loss = 0.5977041, step = 7801 (0.190 sec)
INFO:tensorflow:global_step/sec: 555.853
INFO:tensorflow:loss = 0.53942573, step = 7901 (0.180 sec)
INFO:tensorflow:global_step/sec: 500.256
INFO:tensorflow:loss = 0.5563452, step = 8001 (0.205 sec)
INFO:tensorflow:global_step/sec: 513.106
INFO:tensorflow:loss = 0.55139524, step = 8101 (0.195 sec)
INFO:tensorflow:global_step/sec: 645.505
INFO:tensorflow:loss = 0.577809, step = 8201 (0.155 sec)
INFO:tensorflow:global_step/sec: 588.55
INFO:tensorflow:loss = 0.56536853, step = 8301 (0.165 sec)
INFO:ten

In [22]:
results = estimator.evaluate(input_fn=input_fn,steps=1)

INFO:tensorflow:Starting evaluation at 2018-10-02-14:43:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Lolly\AppData\Local\Temp\tmpgm3m85i5\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-10-02-14:43:13
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.74, accuracy/baseline_label_mean = 0.58, accuracy/threshold_0.500000_mean = 0.74, auc = 0.8197865, auc_precision_recall = 0.8496301, global_step = 10000, labels/actual_label_mean = 0.58, labels/prediction_mean = 0.5731966, loss = 0.5324199, precision/positive_threshold_0.500000_mean = 0.7352941, recall/positive_threshold_0.500000_mean = 0.86206895


In [24]:
print(results)

{'accuracy': 0.74, 'auc': 0.8197865, 'accuracy/baseline_label_mean': 0.58, 'auc_precision_recall': 0.8496301, 'labels/actual_label_mean': 0.58, 'recall/positive_threshold_0.500000_mean': 0.86206895, 'precision/positive_threshold_0.500000_mean': 0.7352941, 'global_step': 10000, 'loss': 0.5324199, 'accuracy/threshold_0.500000_mean': 0.74, 'labels/prediction_mean': 0.5731966}


In [26]:
for var in fit.get_variable_names():
    print(var,"--->",fit.get_variable_value(var))

global_step ---> 10000
linear/bias_weight ---> [0.22031789]
linear/bias_weight/d/linear/bias_weight/part_0/Ftrl ---> [19.418438]
linear/bias_weight/d/linear/bias_weight/part_0/Ftrl_1 ---> [-4.8543034]
linear/x/weight ---> [[19.886522]]
linear/x/weight/head/linear/x/weight/part_0/Ftrl ---> [[0.41695315]]
linear/x/weight/head/linear/x/weight/part_0/Ftrl_1 ---> [[-64.205536]]
